In [1]:
path_to_m3 = '/data/data_geodyn/results/st/msis2/msis2_acceloff/'

In [16]:
# def clean_verify_msis_externally():

import time
start = time.time()



import pandas as pd
msisin_den_file = path_to_m3 + "DENSITY/st030914_2wk.goco05s_msisin"
msisout_den_file = path_to_m3 + "DENSITY/st030914_2wk.goco05s_msisout"
msisSWI_den_file = path_to_m3 + "DENSITY/st030914_2wk.goco05s_msisSWI"

DEN1_csv = pd.read_csv(msisin_den_file, 
                    skiprows = 1, 
                    names = ['IYYDDD',
                             'IYR',
                              'DAY',
                             'UTSEC',
                             'ALTKM',
                             'GLAT',
                             'GLON',
                             'STLOC', 
                             'AVGFLX',
                             'FLUX',
                             'AP1',
                             'AP2',
                             'AP3',
                             'AP4',
                             'AP5',
                             'AP6',
                             'AP7',
                            ],
                    sep = '\s+',
                    )

DEN2_csv = pd.read_csv(msisout_den_file, 
                    skiprows = 1, 
                    names = ['n_HE',
                             'n_O1',
                             'n_N2',
                             'n_O2',
                             'n_Ar',
                             'TotalMassDen',
                             'n_H',
                             'n_N',
                             'n_AnomO',
                             'T_Exo',
                             'T_Alt',
                             'Rho',
                             'dRhodZ',
                                  ],


                    sep = '\s+',
                    )


DEN1_csv['Date'] = (pd.to_datetime('0'+ ((DEN1_csv['IYR'].astype(int).astype(str))),  format='%y') 
                    +  pd.to_timedelta(DEN1_csv['DAY'], unit='days'))


DEN2_csv['Date'] = (pd.to_datetime('0'+ ((DEN1_csv['IYR'].astype(int).astype(str))),  format='%y') 
                    +  pd.to_timedelta(DEN1_csv['DAY'], unit='days'))





with open(msisSWI_den_file, 'r') as f:
    for line_no, line_text in enumerate(f):
        if float(line_text) <= 0:
            print(line_text)

SWI_option = [1.0]*25
SWI_option[8] = -1.0


import matplotlib.pyplot as plt
import numpy as np
from pymsis import msis
import pandas as pd


msis2_df= pd.DataFrame(data={'Rho'   :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'N2'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'O2'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'O'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'He'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'H'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'Ar'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'N'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'AnomO' :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'NO'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                              'Temp'  :np.ones(np.shape(DEN1_csv['Date']))*np.nan})

end1 = time.time()
elapsed1 = end1 - start
print('Start running MSIS: ',elapsed1)

for index, row in DEN1_csv.iterrows():

    lon = row['GLON']
    lat = row['GLAT']
    alts = row['ALTKM']
    f107 = row['FLUX']
    f107a = row['AVGFLX']

    aps = [[row['AP1'],row['AP2'],row['AP3'],row['AP4'],row['AP5'],row['AP6'],row['AP7']]]
    date = row['Date']

    msis_data2 =  msis.run(date, lon, lat, alts, f107, f107a, aps, version=2,  options = SWI_option) #1
    msis_data2 =  msis.run(date, lon, lat, alts, f107, f107a, aps, version=2,  options = SWI_option) #2
    msis_data2 =  msis.run(date, lon, lat, alts, f107, f107a, aps, version=2,  options = SWI_option) #3
    msis_data2 =  msis.run(date, lon, lat, alts, f107, f107a, aps, version=2,  options = SWI_option) #4
    msis_data2 =  msis.run(date, lon, lat, alts, f107, f107a, aps, version=2,  options = SWI_option) #5

    msis_data2  = np.squeeze(msis_data2)

    msis2_df.loc[index, 'Rho'] = msis_data2[0]
    msis2_df.loc[index, 'N2'] = msis_data2[1]
    msis2_df.loc[index, 'O2'] = msis_data2[2]
    msis2_df.loc[index, 'O'] = msis_data2[3]
    msis2_df.loc[index, 'He'] = msis_data2[4]
    msis2_df.loc[index, 'H'] = msis_data2[5]
    msis2_df.loc[index, 'Ar'] = msis_data2[6]
    msis2_df.loc[index, 'N'] = msis_data2[7]
    msis2_df.loc[index, 'AnomO'] = msis_data2[8]
    msis2_df.loc[index, 'NO'] = msis_data2[9]
    msis2_df.loc[index, 'Temp'] = msis_data2[10]

    msis2_df.loc[index, 'Date'] = row['Date']

end = time.time()
elapsed = end - start
print('Stop  running MSIS: ',elapsed,'|  ',elapsed/60,'mins')



    -1.00000000

Start running MSIS:  2.1943883895874023
Stop  running MSIS:  1096.9611148834229 |   18.282685248057046 mins


73.5

# Benchmark Results:

|           Model               |  Arc Length (days) | Step size (secs) | Iterations   | Time (mins)  |  Extrap. Mins |
|-------------------------------|--------------------|------------------|--------------|--------------|---------------|
| MSISe2.0 (`pymsis`)           |     14             |    15            |     1        |   4.741     |               |
| MSISe2.0 (`pymsis`)           |     14             |    15            |     5        | 18.283     |               |
| GEODYN run with MSISe2.0      |     14             |    15            |     5        |  17.491      |               |
|                               |                    |                  |              |              |               |

# Thoughts and Notes


### RR's CTIPe code
- This takes model data output and pulls into kamodo
- takes satellite trajectory and interpolates it to 
- currently takes 4.5 hours

- need a way to return CTIPe object to fortran

- fortran to python 


# ME DO
- see if i can isolate the call to the model so it can be called from python
- see if density call subroutine  can be separated out
- ask if an option exists to grab the density

-------------------------------------------
- everything run from  fortran
- everything run from  -python ------------
- everything run from  fortran
-------------------------------------------


### Notes:
- don't have to run the model in real time... just need the data
- landing in satellite ECEF
- forpy 
- run python in background
- active embedded python session
-----------------------------------------------

- Calling fortran from python....
    - how could this be done

-----------------------------------------------

### Questions:
- possibility of a Kamodo-lite
- this i

In [18]:
DEN1_csv.shape

(321908, 18)

In [12]:
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import plotly.express as px

# col1 = px.colors.qualitative.Plotly[0]
# col2 = px.colors.qualitative.Plotly[1]
# col3 = px.colors.qualitative.Plotly[2]

# mark_size = 3
# fig = make_subplots(rows=1, cols=1, 
# #     subplot_titles=("Density Verification", ),
#                    )

# data_num1 = 1000
# data_num2 = 2


# fig.add_trace(go.Scattergl(x=msis2_df['Date'],
#                          y=msis2_df['Rho'],
#                          name= "Pymsis2",
#                          mode='markers',
#                          marker=dict(color = col2,
#                          size=mark_size,),
#                          ),
#                           row=1, col=1,
#                          )



# fig.update_layout(
#         title="MSIS 2.0 Verification",
#     yaxis_title="Density [kg/m^3]",
#     xaxis_title="Date",
#     )

# fig.update_layout(legend= {'itemsizing': 'constant'})
# fig.update_xaxes(tickangle=30)

# fig.update_yaxes(type="log", exponentformat= 'power')

# fig.show()



KeyError: 'Date'